In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import ray
import time
import tensorflow as tf
from head_to_head_match import head_to_head_match, head_to_head_game
from TrainingManager import TrainingManager
from simulate_self_play_games import simulate_self_play_games

from Config import *

tf.compat.v1.disable_eager_execution()

2023-08-09 08:25:04,012	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
#manager = TrainingManager()
#manager.train()

N = 33

ray.init()

start_time = time.time()

print('Simulating games:')
new_examples = [simulate_self_play_games.remote() for _ in range(N)]
new_examples = ray.get(new_examples)

print(f'Time taken: {time.time() - start_time} seconds')

ray.shutdown()

2023-08-09 08:25:06,316	INFO worker.py:1621 -- Started a local Ray instance.


Simulating games:
(simulate_self_play_games pid=1889164) 1/8, 
(simulate_self_play_games pid=1889167) 4/8,  [repeated 98x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(simulate_self_play_games pid=1889166) 7/8,  [repeated 95x across cluster]
(simulate_self_play_games pid=1889165) 3/8,  [repeated 65x across cluster]
(simulate_self_play_games pid=1889165) 7/8,  [repeated 4x across cluster]
Time taken: 25.920920610427856 seconds
(simulate_self_play_games pid=1889165) 8/8, 
